<a href="https://colab.research.google.com/github/tmoura/softexIA/blob/main/MLflow-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

# Múltiplos Experimentos com Múltiplas Execuções

In [20]:
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

import pandas as pd

url = "https://raw.githubusercontent.com/tmoura/machinelearning/master/datasets/parkinsons.data"

# Carregar base de dados
# DataFrame
dataset = pd.read_csv(url, header=None)

columns = len(dataset.columns)

y = dataset[0] # extrai a primeira coluna, que é o label
X = dataset.loc[:,1:columns-1]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None, stratify=y) # 80% treino e 20% teste

In [ ]:
from sklearn.metrics import confusion_matrix
import mlflow
import mlflow.sklearn

    ######## Experimento 1 ########
exp1 = mlflow.set_experiment(experiment_name="experimento_0")

  ## Run 1 ##
mlflow.start_run(run_name="Run1.1",experiment_id=exp1.experiment_id)
model = tree.DecisionTreeClassifier(criterion="entropy",max_depth=2)
model = model.fit(X_train, y_train)

result = model.predict(X_test)
acc = metrics.accuracy_score(result, y_test)

mlflow.log_param("Criterion", "entropy")
mlflow.log_param("MAX_DEPTH", 2)
mlflow.log_metric("Acurácia", acc)
mlflow.sklearn.log_model(model, "treemodelENTROPY")
mlflow.log_artifact(X)

mlflow.end_run()

  ## Run 2 ##
mlflow.start_run(run_name="Run2.1",experiment_id=exp1.experiment_id)
model = tree.DecisionTreeClassifier()
model = model.fit(X_train, y_train)

result = model.predict(X_test)
acc = metrics.accuracy_score(result, y_test)

mlflow.log_param("Criterion", "gini")
mlflow.log_metric("Acurácia", acc)
mlflow.sklearn.log_model(model, "treemodelGINI")

mlflow.end_run()

######## Experimento 2 ########
exp2 = mlflow.set_experiment(experiment_name="experimento_1")

  ## Run 1 ##
mlflow.start_run(run_name="Run2.1",experiment_id=exp2.experiment_id)
model = KNeighborsClassifier(n_neighbors=5)
model = model.fit(X_train, y_train)

result = model.predict(X_test)
acc = metrics.accuracy_score(result, y_test)

mlflow.log_param("Vizinhos", 5)
mlflow.log_metric("Acurácia", acc)
mlflow.sklearn.log_model(model, "knn5")

mlflow.end_run()

  ## Run 2 ##
mlflow.start_run(run_name="Run2.2",experiment_id=exp2.experiment_id)
model = KNeighborsClassifier(n_neighbors=7)
model = model.fit(X_train, y_train)

result = model.predict(X_test)
acc = metrics.accuracy_score(result, y_test)

mlflow.log_param("Vizinhos", 7)
mlflow.log_metric("Acurácia", acc)
mlflow.sklearn.log_model(model, "knn7")

mlflow.end_run()

get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background

# Uso do NGROK

O Ngrok é uma ferramenta que permite criar um túnel seguro para conectar um servidor local a um servidor remoto, tornando possível expor uma aplicação local para a Internet. Essa ferramenta é muito útil para desenvolvedores que precisam testar suas aplicações localmente antes de colocá-las em produção.

In [25]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2fTc6xBFGHTFjXcla4jSNCgSzt4_78Y6vhwwzuAdc7WFxhTfu"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("Endereço do MLflow:", ngrok_tunnel.public_url)

Endereço do MLflow: https://a842-34-85-213-147.ngrok-free.app
